In [193]:
import nltk
nltk.download('punkt')
import pandas as pd
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [194]:
data = pd.read_csv("ps5_review.csv")
data['flag'] = 0
data = data.drop(['Date', 'Author', 'Headquarters', 'Title', 'Headquarters', 'Review Rating'], axis=1)


TOKENIZATION

In [195]:
from nltk.tokenize import word_tokenize
data['Excerpt'] = data['Excerpt'].apply(word_tokenize)
data['Exceprt'] = data['Excerpt'].str.lower()
print(data.head(21))

             Source                                            Excerpt  flag  \
0         The Verge  [The, PS5, 's, impressive, hardware, and, excl...     0   
1               IGN  [From, the, lightning-fast, SSD, to, the, Dual...     0   
2              CNET  [While, the, PS5, delivers, stunning, visuals,...     0   
3          GameSpot  [The, PS5, 's, launch, lineup, showcases, a, m...     0   
4   Digital Foundry  [In-depth, technical, analysis, of, the, PS5, ...     0   
5            Forbes  [The, PS5, represents, a, significant, leap, i...     0   
6        TechCrunch  [The, haptic, feedback, and, adaptive, trigger...     0   
7         Bloomberg  [The, PS5, 's, powerful, hardware, and, innova...     0   
8          Engadget  [The, PS5, 's, user, interface, is, streamline...     0   
9        GamesRadar  [Comparative, analysis, of, the, PS5, and, Xbo...     0   
10            Wired  [Interview, with, PS5, designers, discussing, ...     0   
11   Tom's Hardware  [An, exploration, o

STEMMING

STOPWORD REMOVAL

In [196]:
from nltk.corpus import stopwords

def stop_it(t):
    dt = [word for word in t if len(word)>2]
    return dt

data['Excerpt'] = data['Excerpt'].apply(stop_it)
print(data.head(21))
data['Excerpt'] = data['Excerpt'].apply(' '.join)
print(data.head(21))

             Source                                            Excerpt  flag  \
0         The Verge  [The, PS5, impressive, hardware, and, exclusiv...     0   
1               IGN  [From, the, lightning-fast, SSD, the, DualSens...     0   
2              CNET  [While, the, PS5, delivers, stunning, visuals,...     0   
3          GameSpot  [The, PS5, launch, lineup, showcases, mix, gen...     0   
4   Digital Foundry  [In-depth, technical, analysis, the, PS5, hard...     0   
5            Forbes  [The, PS5, represents, significant, leap, gami...     0   
6        TechCrunch  [The, haptic, feedback, and, adaptive, trigger...     0   
7         Bloomberg  [The, PS5, powerful, hardware, and, innovative...     0   
8          Engadget  [The, PS5, user, interface, streamlined, and, ...     0   
9        GamesRadar  [Comparative, analysis, the, PS5, and, Xbox, S...     0   
10            Wired  [Interview, with, PS5, designers, discussing, ...     0   
11   Tom's Hardware  [exploration, the, 

SPLITTING

In [197]:
positive_words = ['efficient', 'good', 'impressive', 'forward', 'strengths', 'enhancing', 'innovative', 'powerful', 
'immersive', 'significant', 'capabilities', 'stunning', 'diverse', 'sustainable', 'revolutionizing']

data['flag'] = data.apply(lambda row: 1 if any(word in row['Excerpt'] for word in positive_words) else row['flag'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(data['Excerpt'], data['flag'], test_size=0.25)

print("x_train:")
display(x_train.head())
print("\n")

print("y_train:")
display(y_train.head())


x_train:


0     The PS5 impressive hardware and exclusive game...
19    Analyzing the PS5 energy efficiency and eco-fr...
7     The PS5 powerful hardware and innovative featu...
1     From the lightning-fast SSD the DualSense cont...
3     The PS5 launch lineup showcases mix genres cat...
Name: Excerpt, dtype: object



y_train:


0     1
19    1
7     1
1     1
3     0
Name: flag, dtype: int64

VECTORIZATION

In [198]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer(max_df=0.7)

tfidf_train = my_tfidf.fit_transform(x_train)
tfidf_test = my_tfidf.transform(x_test)
print(tfidf_train)

  (0, 50)	0.17681191153364215
  (0, 17)	0.34562291340828316
  (0, 44)	0.34562291340828316
  (0, 104)	0.34562291340828316
  (0, 99)	0.3001153010529264
  (0, 76)	0.34562291340828316
  (0, 49)	0.24278248991169085
  (0, 36)	0.34562291340828316
  (0, 6)	0.17681191153364215
  (0, 55)	0.2678271362443558
  (0, 62)	0.34562291340828316
  (1, 47)	0.27085921954201503
  (1, 106)	0.311930622174761
  (1, 78)	0.311930622174761
  (1, 19)	0.311930622174761
  (1, 65)	0.311930622174761
  (1, 45)	0.311930622174761
  (1, 31)	0.311930622174761
  (1, 32)	0.311930622174761
  (1, 34)	0.311930622174761
  (1, 5)	0.311930622174761
  (1, 50)	0.15957579035694805
  (1, 6)	0.15957579035694805
  (2, 68)	0.3598055282917344
  (2, 95)	0.3598055282917344
  :	:
  (12, 85)	0.25676092019235974
  (12, 37)	0.2877149864002266
  (13, 83)	0.3269314572957807
  (13, 35)	0.3269314572957807
  (13, 3)	0.3269314572957807
  (13, 16)	0.3269314572957807
  (13, 9)	0.3269314572957807
  (13, 25)	0.3269314572957807
  (13, 4)	0.2838849188626847

LOGISTIC REGRESSION

In [199]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
model_1 = LogisticRegression(max_iter=900)
model_1.fit(tfidf_train, y_train)
predict_1 = model_1.predict(tfidf_test)
accur_pc = accuracy_score(y_test, predict_1)
print(accur_pc*100)

100.0


In [200]:
import numpy as np

# Load the viewership data
viewership_data = pd.read_csv("viewership.csv")

# Specify the unit count of the product
unit_count = 1000  # Change this to your desired unit count

# Filter data with flag == 1
positive_sources = data[data['flag'] == 1]

# Calculate the distribution for each country based on viewership percentages
distribution = {}
for index, row in positive_sources.iterrows():
    source = 'Source'
    source_viewership = viewership_data[viewership_data['Source'] == source]
    
    if not source_viewership.empty:
        viewership_percentage = source_viewership.iloc[0, 1:].sum()  # Sum of viewership percentages for the source
        if not np.isnan(viewership_percentage):  # Check for NaN value
            country_distribution = (source_viewership.iloc[0, 1:] / viewership_percentage) * unit_count
            distribution[source] = country_distribution

# Print the distribution for each source
for source, country_distribution in distribution.items():
    print(f"Distribution for {source}:")
    print(country_distribution)
    print("\n")

